In [17]:
import starepandas
from pyhdf.SD import SD
import numpy
import pystare
import xarray
import dask
import datetime

In [18]:
client = dask.distributed.Client(n_workers=4)

/home/griessbaum/.virtualenvs/starepandas/lib/python3.8/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 44345 instead
  warnings.warn(


In [19]:
file_path = '/home/griessbaum/MOD09/MOD09.A2020187.1740.006.2020189022420.hdf'
hdf = SD(file_path)
lon = hdf.select('Longitude').get().astype(numpy.double)
lat = hdf.select('Latitude').get().astype(numpy.double)

HDF4Error: SD (60): HDF Internal error

In [6]:
start = datetime.datetime.now()
stare = pystare.from_latlon2D(lat=lat, 
                              lon=lon, 
                              adapt_resolution=True)
print(datetime.datetime.now()-start)

0:00:25.981664


# Dask

In [9]:
coords = numpy.array([lat, lon])
coords_d = dask.array.from_array(coords, chunks=(2,500,1354))
coords_d

dask.array<array, shape=(2, 2030, 1354), dtype=float64, chunksize=(2, 500, 1354), chunktype=numpy.ndarray>

In [ ]:
def stare(coords):
    return pystare.from_latlon2D(coords[0], 
                                 coords[1], adapt_resolution=True)

In [ ]:
s_d = coords_d.map_blocks(stare, drop_axis=[0], 
                            chunks=(100, 1354), dtype='int64')
s_d = s_d.compute()

# Xarray Ufunc

In [8]:
lat_x = xarray.DataArray(lat, dims=['x', 'y']).chunk({'x': 500})
lon_x = xarray.DataArray(lon, dims=['x', 'y']).chunk({'x': 500})

In [9]:
start = datetime.datetime.now()
s_d = xarray.apply_ufunc(pystare.from_latlon2D, 
                         lat_x,
                         lon_x,
                         dask='parallelized',
                         output_dtypes=[numpy.int64])

sids = numpy.array(s_d)
print(datetime.datetime.now()-start)

0:00:13.523768


In [10]:
sids

array([[3323448502995910555, 3323477456837767419, 3323478059551377211,
        ..., 3139094890924297243, 3139093901117727963,
        3139094674461717019],
       [3323448559446177211, 3323476481434354011, 3323476574637988187,
        ..., 3139095249270340955, 3139093924786339323,
        3139094635810779387],
       [3323449598137104443, 3323447781968162363, 3323476531348222523,
        ..., 3139094794814830779, 3139094081449199835,
        3139093808164983259],
       ...,
       [3446185661822640251, 3446184725169396443, 3446178856607724667,
        ..., 3396773703334098843, 3396772137745922075,
        3396492751100275739],
       [3446186011939297659, 3446184633907673723, 3446184490778651099,
        ..., 3396773741939207547, 3396492712355875003,
        3396494589764107579],
       [3446185890551234171, 3446184977672977307, 3446184512045768475,
        ..., 3396773719347289211, 3396493429055275259,
        3396494541816577659]])

# Write Sidecar

In [16]:
import netCDF4
rootgrp = netCDF4.Dataset('test.nc', "w", format="NETCDF4")

rootgrp.close()

# Dask DataFrame

In [12]:
band1 = hdf.select('1km Surface Reflectance Band 1').get().astype(numpy.double)
#lat = hdf.select('Latitude').get().astype(numpy.double)
band1

array([[ 356.,  400.,  772., ..., 1233., 1070., 1072.],
       [ 308.,  324.,  499., ..., 1306.,  930., 1464.],
       [ 337.,  320.,  341., ..., 1031., 1009., 1863.],
       ...,
       [ 951., 1234., 1786., ..., 8439., 8035., 7901.],
       [ 901., 1094., 1574., ..., 8482., 8176., 8034.],
       [ 818.,  970., 1426., ..., 8454., 8229., 8062.]])

In [13]:
import pandas
df = pandas.DataFrame({'stare': sids, 'band1': band1.flatten()})
ddf = dask.dataframe.from_pandas(df, npartitions=4)
ddf.set_index('stare')

,band1
npartitions=4,
3064699558929323008,float64
3333228369467680768,...
3338832772939591680,...
3348567057403807744,...
3450320257242284032,...
